# Dataset and Preprocessing

At first, our team agreed that each member would identify at least one dataset with rich enough variables to yield multiple analytical perspectives. During our kickoff meeting, we evaluated several sources and discussed potential correlations. Ultimately, we selected the **Our World in Data CO₂ Emissions** and **WHO Healthy Life Expectancy (HALE)** datasets, spanning the years **2000 through 2021**, because they align perfectly with our main research question:

> **How do national CO₂ emissions relate to healthy life expectancy across countries and over time?**

These datasets provide a complementary view—environmental impact vs. population health—and include consistent identifiers (`iso_code`, `country`, `year`) for reliable merging.

## Cleaning

Both OWID and WHO datasets undergo periodic schema changes (column renames, extra metadata). We adopted a **two-phase cleaning process**:

1. **Rename & restructure**  
   - Standardize column names:
     - `Country` → `country`
     - `Year` → `year`
     - `Healthy_life_expectancy_at_birth` → `healthy_life_expectancy`
   - Select only the necessary fields:
     - CO₂: `iso_code`, `country`, `year`, `co2_per_capita`, `cumulative_co2`
     - HALE: `country`, `year`, `healthy_life_expectancy`

2. **Filter & normalize**  
   - Retain five **reference years**: 2000, 2010, 2015, 2019, 2021.
   - Exclude aggregate regions (e.g., “World”, “Asia”, “European Union (27)”) to focus on individual countries.
   - Normalize inconsistent country names via ISO codes where needed.

To optimize performance and storage, we save the final cleaned dataset as a **Parquet** file with Gzip compression, reducing file size significantly.

## Variable Descriptions

In the cleaned dataset:

- **Continuous / Ratio variables:**
  - `co2_per_capita`: Tonnes of CO₂ per person, per year
  - `cumulative_co2`: Total tonnes of CO₂ emitted up to that year
  - `healthy_life_expectancy`: Expected years of life in full health

- **Categorical / Discrete variables:**
  - `country`: Country name
  - `year`: Reference year (integer)

These fields form the core of our analysis.

## Custom Aggregation: CO₂ Intensity per Healthy Year

To relate emissions directly to health outcomes, we define:

$$
\textrm{CO2Intensity}
=
\frac{\textrm{co2_per_capita}}
     {\textrm{healthy_life_expectancy}}
$$


This indicates how many tonnes of CO₂ are emitted for each expected year of healthy life.

**Example:**
- `co2_per_capita = 8.0` tonnes  
- `healthy_life_expectancy = 72.0` years  
- CO₂ Intensity ≈ 0.11 t CO₂ per healthy year  



**Example:**  
- `co2_per_capita = 8.0` tonnes  
- `healthy_life_expectancy = 72.0` years  
- CO₂ Intensity ≈ 0.11 t CO₂ per healthy year

In [ ]:
import pandas as pd

# 1. Load datasets
co2 = pd.read_csv('owid-co2-data.csv')
hale = pd.read_csv('C64284D_ALL_LATEST.csv')

# 2. Select years & fields
years = [2000, 2010, 2015, 2019, 2021]
co2_sel = co2[co2['year'].isin(years)][['iso_code','country','year','co2_per_capita','cumulative_co2']].copy()
hale_sel = (hale[hale['Year'].isin(years)]
            [['Country','Year','Healthy_life_expectancy_at_birth']]
            .rename(columns={'Country':'country','Year':'year','Healthy_life_expectancy_at_birth':'healthy_life_expectancy'})
            .copy())

# 3. Exclude aggregates
aggregates = ['World','Asia','Europe','Africa','North America','South America','Oceania','European Union (27)']
co2_clean = co2_sel[~co2_sel['country'].isin(aggregates)]
hale_clean = hale_sel[~hale_sel['country'].isin(aggregates)]

# 4. Merge and clean
merged_df = pd.merge(co2_clean, hale_clean, on=['country','year'], how='inner').dropna()

# 5. Calculate CO₂ Intensity
merged_df['co2_intensity'] = merged_df['co2_per_capita'] / merged_df['healthy_life_expectancy']

# 6. Inspect
merged_df.head()